# In order

In [293]:
import pandas as pd
import numpy as np
import time
import requests
import re
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import pickle
from tqdm import tqdm

## Thinking
Go back to when All-Star teams were first chosen by fan voting

# Getting All-Star votes data


In [42]:
#ua = UserAgent()
#user_agent = {'User-agent': ua.random}
#url = "https://www.basketball-reference.com/allstar/"
#response = requests.get(url, headers=user_agent)
#print("Request status code: {}".format(str(response.status_code)))
#page = response.text
#soup = BeautifulSoup(page)

In [193]:
#links = []
#table = soup.find("table", id="all_star_games_nba")
#rows = table.find_all("tr")
#Next("tbody")
#rows = body.find_all("tr")
#body
#for row in rows:
#    for item in row.find_all("td"):
#        if item.findNext("a").text == "Voting":
#            links.append("https://www.basketball-reference.com{}".format(item.findNext("a")["href"]))
#links

In [188]:
# Master function

def get_all_star_votes(list_of_years):
    '''
    Arguments: List
    Returns: Dictionary
    '''
    all_star_votes = {}
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}

    for year in list_of_years:
        if year != 1999:
            year_all_stars_dict = {}
            url = "https://www.basketball-reference.com/allstar/NBA_{}_voting.html".format(year)
            response = requests.get(url, headers=user_agent)
            print("{} request status code: {}".format(str(year), str(response.status_code)))
            page = response.text
            soup = BeautifulSoup(page)
            whole_set = soup.find_all("div", class_="data_grid_group solo")

            # Getting starters
            starters = soup.find("div", class_="data_grid_group solo").find_all("div", id=re.compile("(all-star-starters).+"))
            starters_dict = {}
            for team in starters:
                conference = team.find("table").caption.text[:4]
                conference_dict = {}
                players = team.find("table").find_all("tr")
                for player in players:
                    player_dict = {}
                    player_details = player.find_all("td")
                    if len(player_details) == 3:
                        position = player_details[0].text
                        name = player_details[1].text
                        votes = int("".join(player_details[2].text.split(",")))
                        stats_page = "https://www.basketball-reference.com{}".format(player_details[1].a["href"])
                        conference_dict[name] = {"Position": position, "Votes": votes, "Player page": stats_page}
                year_all_stars_dict["{} starters".format(conference)] = conference_dict

            # Getting East votes
            east = whole_set[1]
            east_dict = {}
            for table in east.find_all("table"):
                position = table.caption.text
                players = table.find_all("tr")
                for player in players:
                    player_dict = {}
                    player_details = player.find_all("td")
                    if len(player_details) == 3:
                        name = player_details[1].text
                        votes = int("".join(player_details[2].text.split(",")))
                        stats_page = "https://www.basketball-reference.com{}".format(player_details[1].a["href"])
                        if year in [2013, 2014, 2015, 2016]:
                            if position == "Backcourt":
                                position = "BC"
                            else:
                                position = "FC"
                        east_dict[name] = {"Position": position, "Votes": votes, "Player page": stats_page}
            year_all_stars_dict["East full"] = east_dict

            # Getting West votes
            west = whole_set[2]
            west_dict = {}
            for table in east.find_all("table"):
                position = table.caption.text
                players = table.find_all("tr")
                for player in players:
                    player_dict = {}
                    player_details = player.find_all("td")
                    if len(player_details) == 3:
                        name = player_details[1].text
                        votes = int("".join(player_details[2].text.split(",")))
                        stats_page = "https://www.basketball-reference.com{}".format(player_details[1].a["href"])
                        if year in [2013, 2014, 2015, 2016]:
                            if position == "Backcourt":
                                position = "BC"
                            else:
                                position = "FC"
                        west_dict[name] = {"Position": position, "Votes": votes, "Player page": stats_page}
            year_all_stars_dict["West full"] = west_dict

            # Adding year to larger dict and pickling
            all_star_votes[year] = year_all_stars_dict
            all_star_votes_df = pd.DataFrame.from_dict(all_star_votes, orient="index")
            with open("all_star_votes.pickle", "wb") as to_write:
                pickle.dump(all_star_votes_df, to_write)
            print("Just pickled!")

    return all_star_votes_df


In [189]:
all_star_votes_df = get_all_star_votes([i for i in range(1975, 2017)])

1975 request status code: 200
Just pickled!
1976 request status code: 200
Just pickled!
1977 request status code: 200
Just pickled!
1978 request status code: 200
Just pickled!
1979 request status code: 200
Just pickled!
1980 request status code: 200
Just pickled!
1981 request status code: 200
Just pickled!
1982 request status code: 200
Just pickled!
1983 request status code: 200
Just pickled!
1984 request status code: 200
Just pickled!
1985 request status code: 200
Just pickled!
1986 request status code: 200
Just pickled!
1987 request status code: 200
Just pickled!
1988 request status code: 200
Just pickled!
1989 request status code: 200
Just pickled!
1990 request status code: 200
Just pickled!
1991 request status code: 200
Just pickled!
1992 request status code: 200
Just pickled!
1993 request status code: 200
Just pickled!
1994 request status code: 200
Just pickled!
1995 request status code: 200
Just pickled!
1996 request status code: 200
Just pickled!
1997 request status code: 200
Ju

In [231]:
# Getting rosters

def get_all_star_rosters(list_of_years):

    all_star_rosters = {}
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}

    for year in list_of_years:
        if year != 1999:
            url = "https://www.basketball-reference.com/allstar/NBA_{}.html".format(year)
            new_response = requests.get(url, headers=user_agent)
            print("Request status code: {}".format(str(new_response.status_code)))
            new_page = new_response.text
            soup = BeautifulSoup(new_page)

            player_list = []
            rosters = soup.find_all("div", class_="overthrow table_container")
            for table in rosters:
                for player in table.find_all("th", scope="row"):
                    player_name = player.find("a")
                    if player_name != None:
                        player_list.append(player_name.text)

            injury_notes = soup.find_all("ul", class_="page_index")

            # Getting injury notes
            for injury in injury_notes:
                player_name = soup.find("ul", class_="page_index").find("a").text.strip()
                note = injury.find("div").text.strip()
                player_list.append("{} ({})".format(player_name, note))

            all_star_rosters[year] = player_list
            all_star_rosters_df = pd.DataFrame.from_dict(all_star_rosters, orient="index")
            with open("all_star_rosters.pickle", "wb") as to_write:
                    pickle.dump(all_star_rosters_df, to_write)
            print("Just pickled!")

    return all_star_rosters_df

In [232]:
all_star_rosters_df = get_all_star_rosters([i for i in range(1975, 2020)])

Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200
Just pickled!
Request status code: 200


In [ ]:
# Getting All-Star Selections

def get_all_star_votes(list_of_years):
    '''
    Arguments: List
    Returns: Dictionary
    '''
    all_star_rosters = {}
    ua = UserAgent()
    user_agent = {'User-agent': ua.random}

    for year in list_of_years:
        if year != 1999:
            year_all_stars_dict = {}
            url = "https://www.basketball-reference.com/allstar/NBA_{}.html".format(year)
            response = requests.get(url, headers=user_agent)
            print("{} request status code: {}".format(str(year), str(response.status_code)))
            page = response.text
            soup = BeautifulSoup(page)
            whole_set = soup.find_all("div", class_="data_grid_group solo")

            # Getting starters
            starters = soup.find("div", class_="data_grid_group solo").find_all("div", id=re.compile("(all-star-starters).+"))
            starters_dict = {}
            for team in starters:
                conference = team.find("table").caption.text[:4]
                conference_dict = {}
                players = team.find("table").find_all("tr")
                for player in players:
                    player_dict = {}
                    player_details = player.find_all("td")
                    if len(player_details) == 3:
                        position = player_details[0].text
                        name = player_details[1].text
                        votes = int("".join(player_details[2].text.split(",")))
                        stats_page = "https://www.basketball-reference.com{}".format(player_details[1].a["href"])
                        conference_dict[name] = {"Position": position, "Votes": votes, "Player page": stats_page}
                year_all_stars_dict["{} starters".format(conference)] = conference_dict

            # Getting East votes
            east = whole_set[1]
            east_dict = {}
            for table in east.find_all("table"):
                position = table.caption.text
                players = table.find_all("tr")
                for player in players:
                    player_dict = {}
                    player_details = player.find_all("td")
                    if len(player_details) == 3:
                        name = player_details[1].text
                        votes = int("".join(player_details[2].text.split(",")))
                        stats_page = "https://www.basketball-reference.com{}".format(player_details[1].a["href"])
                        if year in [2013, 2014, 2015, 2016]:
                            if position == "Backcourt":
                                position = "BC"
                            else:
                                position = "FC"
                        east_dict[name] = {"Position": position, "Votes": votes, "Player page": stats_page}
            year_all_stars_dict["East full"] = east_dict

            # Getting West votes
            west = whole_set[2]
            west_dict = {}
            for table in east.find_all("table"):
                position = table.caption.text
                players = table.find_all("tr")
                for player in players:
                    player_dict = {}
                    player_details = player.find_all("td")
                    if len(player_details) == 3:
                        name = player_details[1].text
                        votes = int("".join(player_details[2].text.split(",")))
                        stats_page = "https://www.basketball-reference.com{}".format(player_details[1].a["href"])
                        if year in [2013, 2014, 2015, 2016]:
                            if position == "Backcourt":
                                position = "BC"
                            else:
                                position = "FC"
                        west_dict[name] = {"Position": position, "Votes": votes, "Player page": stats_page}
            year_all_stars_dict["West full"] = west_dict

            # Adding year to larger dict and pickling
            all_star_votes[year] = year_all_stars_dict
            all_star_votes_df = pd.DataFrame.from_dict(all_star_votes, orient="index")
            with open("all_star_votes.pickle", "wb") as to_write:
                pickle.dump(all_star_votes_df, to_write)
            print("Just pickled!")

    return all_star_votes_df


## Getting TV market data

In [296]:
for i in tqdm(range(10000000)):
    pass

100%|██████████| 10000000/10000000 [00:02<00:00, 3603416.18it/s]


In [388]:
# Getting TV market size in 2000-01 and 2001-02 (except in Canada)
url = "https://www.sportsbusinessdaily.com/Daily/Issues/2001/08/14/Ratings-Research/NIELSEN-DMA-RANKINGS.aspx"
response = requests.get(url, headers=user_agent)
print("Request status code: {}".format(str(response.status_code)))
page = response.text
soup = BeautifulSoup(page)

table = soup.find("article", class_="article normal row u-vr4").find_all("table")[1]
markets = table.find_all("tr")[1:]
market_dict = {}
for market in markets:
    market_details = market.find_all("td")
    city_as_list = [word.strip() for word in market_details[2].text.split("      ")]
    if city_as_list.count("") > 0:
        city_as_list.remove("")
    name = " ".join(city_as_list)
    size01 = int("".join(market_details[3].text.split(',')).strip())
    size02 = int("".join(market_details[4].text.split(',')).strip())
    market_dict[name] = [size01, size02]

# Matching teams to markets
url = "https://www.sportsmediawatch.com/nba-market-size-nfl-mlb-nhl-nielsen-ratings/"
response = requests.get(url, headers=user_agent)
print("Request status code: {}".format(str(response.status_code)))
page = response.text
soup = BeautifulSoup(page)

market_team_dict = {}
markets_teams = soup.find("tbody").find_all("tr")
for market_team in markets_teams[1:]:
    rows = market_team.find_all("td")
    teams = rows[3].text
    if teams != "no team":
        market = rows[1].text
        if market == "New YorkCity":
            market = "New York"
        elif market == "Washington D.C.":
            market = "Washington, DC (Hagerstown)"
        elif market == "Cleveland-Akron":
            market = "Cleveland"
        elif market == "Portland":
            market = "Portland, OR"
        elif market == "Boston":
            market = "Boston (Manchester)"
        elif market == "Orlando-Daytona":
            market = "Orlando-Daytona Beach-Melborne"
        if "," in teams:
            team_list = teams.split(",")
            for team in team_list:
                market_team_dict[team] = market
        else:
            market_team_dict[teams] = market

# Getting team abbreviations (which are in main dataset) and adding those missing (see below)
url = "https://en.wikipedia.org/wiki/Wikipedia:WikiProject_National_Basketball_Association/National_Basketball_Association_team_abbreviations"
response = requests.get(url, headers=user_agent)
print("Request status code: {}".format(str(response.status_code)))
page = response.text
soup = BeautifulSoup(page)

team_abbrev_dict = {}
teams_abbrevs = soup.find("table").find_all("tr")
for team in teams_abbrevs[1:]:
    abbrev = team.find_all("td")[0].text.strip()
    team_name = team.find_all("td")[1].text.strip()
    team_abbrev_dict[abbrev] = team_name

# Adding: TOR, CHH, CHO, VAN, NOK, NJN, SEA, NOH
team_abbrev_dict["CHH"] = "Charlotte Hornets"
team_abbrev_dict["CHO"] = "Charlotte Hornets"
team_abbrev_dict["NOH"] = "New Orleans Hornets"
team_abbrev_dict["NOK"] = "New Orleans/Oklahoma City Hornets"
team_abbrev_dict["NJN"] = "New Jersey Nets"
team_abbrev_dict["SEA"] = "Seattle Supersonics"
team_abbrev_dict["TOR"] = "Toronto Raptors"
team_abbrev_dict["VAN"] = "Vancouver Grizzlies"

# Compiling this all so far in one dict with abbreviations as keys
new_dict = {}
for key, value in team_abbrev_dict.items():
    if key == 'NOH':
        new_dict["NOH"] = {"Full name": "New Orleans Hornets", "Market": "New Orleans"}
    elif key == "NOK":
        new_dict["NOK"] = {"Full name": "New Orleans/Oklahoma City Hornets", "Market": "Oklahoma City"}
    elif key == "TOR":
        new_dict["TOR"] = {"Full name": "Toronto Raptors", "Market": "Toronto"}
    elif key == "VAN":
        new_dict["VAN"] = {"Full name": "Vancouver Grizzlies", "Market": "Vancouver"}
    else:
        for key2, value2 in market_team_dict.items():
            if key2 in value:
                new_dict[key] = {"Full name": value, "Market": value2}

# Adding market size to create full
full_team_dict = {}
for key, value in tqdm(new_dict.items()):
    if value["Market"] in market_dict:
        value_dict = value
        value_dict["2000-01 TV market size"] = market_dict[value["Market"]][0]
        value_dict["2001-02 TV market size"] = market_dict[value["Market"]][1]
        full_team_dict[key] = value_dict
    else:
        value_dict = value
        value_dict["2000-01 TV market size"] = np.NaN
        value_dict["2001-02 TV market size"] = np.NaN
        full_team_dict[key] = value_dict
full_team_dict

Request status code: 200
Request status code: 200
100%|██████████| 35/35 [00:00<00:00, 147538.33it/s]Request status code: 200



{'ATL': {'Full name': 'Atlanta Hawks',
  'Market': 'Atlanta',
  '2000-01 TV market size': 1857220,
  '2001-02 TV market size': 1990650},
 'BKN': {'Full name': 'Brooklyn Nets',
  'Market': 'New York',
  '2000-01 TV market size': 6935610,
  '2001-02 TV market size': 7301060},
 'BOS': {'Full name': 'Boston Celtics',
  'Market': 'Boston (Manchester)',
  '2000-01 TV market size': 2242240,
  '2001-02 TV market size': 2315700},
 'CHA': {'Full name': 'Charlotte Hornets',
  'Market': 'Charlotte',
  '2000-01 TV market size': 903950,
  '2001-02 TV market size': 954210},
 'CHI': {'Full name': 'Chicago Bulls',
  'Market': 'Chicago',
  '2000-01 TV market size': 3244850,
  '2001-02 TV market size': 3360770},
 'CLE': {'Full name': 'Cleveland Cavaliers',
  'Market': 'Cleveland',
  '2000-01 TV market size': 1488270,
  '2001-02 TV market size': 1513130},
 'DAL': {'Full name': 'Dallas Mavericks',
  'Market': 'Dallas-Ft. Worth',
  '2000-01 TV market size': 2069010,
  '2001-02 TV market size': 2201170},
 'D